In [ ]:
### THIS MAY NOT BE NEEDED FOR DEPLOY SINCE FILES ARE ON GITHUB
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install emoji

     |████████████████████████████████| 185 kB 5.4 MB/s 
  Created wheel for emoji: filename=emoji-1.4.1-py3-none-any.whl size=186393 sha256=212ef907e46cfdb77089231441590bc7527eda63f8fb678b49263892dd41b444
  Stored in directory: /root/.cache/pip/wheels/8e/68/ac/537456a5331f1405779f2b3c2a578429d2f6d7419e440330d8
Successfully built emoji


In [10]:
import numpy as np
import torch
import pickle
import re
from emoji import demojize
import string
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading Trained weights and vocabulary

In [11]:
### CHANGE TO GITHUB REPO WHERE STREAMLIT FILES ARE 
datapath = '/content/drive/MyDrive/Omdena-India-Socio-Economy/'
fmodel = datapath + 'LSTM_RNN_Sentiment_model.pt'
fvocab = datapath + 'LSTM_RNN_Sentiment_vocab.pkl'

In [12]:
print(fmodel)

/content/drive/MyDrive/Omdena-India-Socio-Economy/LSTM_RNN_Sentiment_model.pt


In [13]:
### Load Trained Vocabulary
def load_vocab(fpath):
  pickled_data = open(fpath, 'rb')
  vocab = pickle.load(pickled_data)
  return vocab

vocab_to_int = load_vocab(fvocab)

In [14]:
vocab_to_int

{'covid': 1,
 'corona': 2,
 'india': 3,
 'lockdown': 4,
 'cases': 5,
 'people': 6,
 'pandemic': 7,
 'sir': 8,
 'please': 9,
 'time': 10,
 'govt': 11,
 'us': 12,
 'help': 13,
 'one': 14,
 'positive': 15,
 'due': 16,
 'day': 17,
 'new': 18,
 'like': 19,
 'virus': 20,
 'home': 21,
 'take': 22,
 'get': 23,
 'today': 24,
 'students': 25,
 'also': 26,
 'government': 27,
 'situation': 28,
 'world': 29,
 'state': 30,
 'need': 31,
 'days': 32,
 'good': 33,
 'health': 34,
 'work': 35,
 'fight': 36,
 'coronavirus': 37,
 'may': 38,
 'country': 39,
 'many': 40,
 'pm': 41,
 'total': 42,
 'social': 43,
 'even': 44,
 'via': 45,
 'life': 46,
 'well': 47,
 'patients': 48,
 'hospital': 49,
 'news': 50,
 'delhi': 51,
 'deaths': 52,
 'exams': 53,
 'know': 54,
 'last': 55,
 'modi': 56,
 'go': 57,
 'stay': 58,
 'app': 59,
 'care': 60,
 'family': 61,
 'spread': 62,
 'let': 63,
 'first': 64,
 'vaccine': 65,
 'indian': 66,
 'distancing': 67,
 'still': 68,
 'year': 69,
 'ji': 70,
 'going': 71,
 'see': 72,
 'safe

### Sentiment Model

In [15]:
### SENTIMENT MODEL 
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

### Functions to pre-process input text

In [16]:
def clean_text(text):
    # Convert Emoji to strings
    #text = demojize(text)

    # Remove HTML Tags
    try:
        text = html.document_fromstring(text).text_content()
    except:
        pass
          
    # Remove Hyperlinks
    text = re.sub('http\S+', ' ', text)
    
    # Remove html character codes ( like &amp)
    text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', ' ', text)
    
    # Remove Hashtags
    text = re.sub("#([a-zA-Z0-9_]{1,50})", ' ', text)
    
    # Remove Mentions
    text = re.sub("@([a-zA-Z0-9_]{1,50})", ' ', text)

    # Remove non alphabets
    text = re.sub('[^a-zA-Z ]+', ' ', text)    
    
    # Remove punctuations
    text = re.sub('[^-9A-Za-z ]', ' ' , text)

    # Lowercase and split - converts to list
    text = text.lower().split()  
    
    # Remove stop and short words
    text = [word for word in text if word not in stop and len(word) > 1]
    
    # Stemming
    # text = [stemmer.stem(word) for word in text]

    # Join and Return
    return ' '.join(text )

def tokenize_text(text):

    # splitting by spaces
    test_words = text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words])

    return test_ints


In [21]:
def pad_features(test_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(test_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(test_ints):
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

### Instantiating Model

In [22]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(60399, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


### Load pre-trained weights

In [23]:
# Load Trained Model weights
model = net.load_state_dict(torch.load(fmodel, map_location=torch.device('cpu')))
print(model)

<All keys matched successfully>


### Call Function

In [33]:
def predict(net, test_text, sequence_length=200):
    
    net.eval()
    
    # tokenize text
    text = clean_text(test_text)
    test_ints = tokenize_text(text)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
  
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    
    if(pred.item()==1):
        response = "Positive review detected!"
    else:
        response = "Negative review detected."
    
    pred_value = output.item()

    return response, pred_value


## Example Input and Output

In [32]:
input_text = "people happy"
seq_length=51 # good to use the length that was trained on
response, pred_value = predict(net, input_text, seq_length)
print(response)
print('Prediction value, pre-rounding: {:.6f}'.format(pred_value))

Positive review detected!
Prediction value, pre-rounding: 0.998327


In [34]:
input_text = "covid was devastating"
seq_length=51 # good to use the length that was trained on
response, pred_value = predict(net, input_text, seq_length)
print(response)
print('Prediction value, pre-rounding: {:.6f}'.format(pred_value))

Negative review detected.
Prediction value, pre-rounding: 0.000275
